# Introduction to TensorFlow 
---

Code and examples have been extracted from the following sources
 - Hands-On Machine Learning with Scikit-Learn and TensorFlow: Concepts, Tools, and Techniques to Build Intelligent Systems, Géron A, 2017

- Natural Language Processing with TensorFlow Teach language to machines using Python's deep learning library, Ganegedara T, 2018


---

### Introduction

Tensorflow is an open source library developed by Google Brain in order to perform numerical computations. Particularly, it is useful for performing large scale automatic differentiation.

Its logic is simple: first it defines a computational graph, then, Tensorflow distributes and runs the operations efficiently using optimized C++ code. 

Tensorflow supports distributed computations---multiple cpus, multiple gpus---therefore, it can train neural nets with millions of parameters in a reasonable amount of time. 

Tensorflow's source code is available at:  [Tensorflow project repository](https://github.com/tensorflow/tensorflow)


### Components 

* **Graph** This object contains the computational graph that connects the input and output nodes across the different operations to execute. The graph determines the flow and order of data. It stores relevant information such as the dependencies of each node and their respective order and place. 

* **Session** The session is in charge of starting the execution graph. Behind the scenes, the session sends the graph definition `tf.GraphDef` as an execution protocol buffer to the distributed *Master*. 

* **Master** The master surveys the graph, it subdivies it into smaller tasks---made up of instructions and operations---and distribute them across *worker* nodes. 

* **Worker** Are the nodes in charge of executing the tasks assigned by the *Master*; there are two types of workers:

     * **Operation Server** It's the node in charge of performing the assigned operations. 
     * **Parameter Server** These nodes store the necessary parameters for each operation and update their status after these have been performed. 
    

![alt text](./imgs/execTF.png "Ejecución proceso en TF")


![alt text](./imgs/operationParameter.png "Nodos de operación y de parámetros")

### Cafe Metaphor

In order to understand the role of each component within Tensorflow, it is useful to think about a restaurant or café. Let us suppose that we are about to order a hamburger in a restaurant. The hamburger we have in mind is the equivalent to the computational graph. The waiter is the equivalent to the session: he knows what should be executed and registers the specifics of the order. In this sense, the order represents the graph definition *GraphDef*. The kitchen manager, the *Master*, takes the order and assigns the different tasks to the different chefs and cooks. The chef (operation node) tells the cook what is needed to be done and the cook (the parameter node) looks for the ingredients in order for the chef to have them nearby. 


![alt text](./imgs/cafeteria.png "Operation nodes and parameter nodes")

### Logistic example

Import libraries




In [1]:
import numpy as np
import tensorflow as tf

Initialize graph and session

In [2]:
graph = tf.Graph() 
session = tf.InteractiveSession(graph=graph) 

Define a *placeholder* as a node for the introduction of inputs

In [3]:
x = tf.placeholder(shape=[1,10], dtype=tf.float32, name='x') 

Define variables to perform operations

In [4]:
W = tf.Variable(tf.random_uniform(shape=[10,5], minval=-0.1, 
                                  maxval=0.1, 
                                  dtype=tf.float32), 
                name='W')
b = tf.Variable(tf.zeros(shape=[5], dtype=tf.float32),
                name='b')  
h = tf.nn.sigmoid(tf.matmul(x, W) + b)

Initialize variables and start node execution

In [5]:
tf.global_variables_initializer().run()

Perform operations supplying the value of x for the session

In [6]:
h_eval = session.run(h, feed_dict={x: np.random.rand(1, 10)})

In [7]:
print(h_eval)

[[0.5053216  0.466581   0.51042956 0.50397724 0.46911782]]


In [8]:
session.close()

![alt text](./imgs/variableTypes.png "Nodos de operación y de parámetros")

### Starting over from scratch

It is critical to understand the role of each component within the execution graph. Thus in the following sections we'll carry out some examples with increasing complexity until we get to build complex neural nets. Thus let us begin by computing the following function: 


<center>$f(x) = x^2y + y + 2$</center>


Variable definition

In [9]:
x = tf.Variable(3, name="x" ) 
y = tf.Variable(4, name="y" ) 
f = x*x*y + y + 2

It is important to remember that this code doesn't perform any operation yet. The variables haven't even been initialized. In order to do this, we need to instantiate a session. 

In [10]:
sess = tf.Session() 
sess.run(x.initializer)
sess.run(y.initializer)

Now we can execute the task

In [11]:
result = sess.run(f)
print(result)

42


In [12]:
sess.close()

If we want to avoid calling each initializer separately:

In [ ]:
init = tf.global_variables_initializer()
with tf.Session() as sess:
    init.run()
    result = f.eval()  # sess.run(f) 
    print(result)

This code is equivalent to the follwing:

In [13]:
with tf.Session() as sess:
    x.initializer.run()  # same as tf.get_default_session().run(x.initializer)
    y.initializer.run()  # same as tf.get_default_session().run(y.initializer)
    result = f.eval()  # same as tf.get_default_session().run(f)
    print(result)

42


In [14]:
x = tf.Variable([2, 1], dtype=tf.float32, name='x')
y = tf.Variable([4, 3], dtype=tf.float32, name='y')
z = tf.multiply(x, y)
init = tf.global_variables_initializer()
with tf.Session() as sess:
    init.run()
    result = z.eval()
    print(result)

[8. 3.]


### Graph scope

In [16]:
# We create two graphs
graph1 = tf.Graph()
graph2 = tf.Graph()
# A session per graph
with tf.Session(graph=graph1) as sess1:
    x1 = tf.Variable(3)
    
with tf.Session(graph=graph2) as sess2:
    x2 = tf.Variable(4)
    
print('''x1 in graph1: {val}'''.format(val=x1.graph is graph1))
print('''x2 in graph2: {val}'''.format(val=x2.graph is graph2))
print('''x1 in graph2: {val}'''.format(val=x1.graph is graph2))

x1 in graph1: True
x2 in graph2: True
x1 in graph2: False


In [17]:
w = tf.constant(4)
x = 5 + w
y = x - 7
z = y * x

In [18]:
with tf.Session() as sess: 
    print(y.eval())
    print(z.eval())

2
18


The code is executed two times since the value of each node needs to be computed for each graph execution. In order to carry out this operation efficiently we need to compute z and y over the same graph. 

In [19]:
with tf.Session() as sess:
    y_, z_ = sess.run([y, z])
    print(y_)
    print(z_)

2
18


### Basic operations



In [22]:
graph = tf.Graph()
sess = tf.InteractiveSession(graph=graph)

/anaconda3/envs/nlpwts/lib/python3.6/site-packages/tensorflow/python/client/session.py:1645: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


two simple tensors

In [23]:
x = tf.constant([[1, 2], [3, 4]], dtype=tf.float32)
y = tf.constant([[4, 3], [3, 2]], dtype=tf.float32)

In [24]:
x_plus_y = tf.add(x, y)
x_mul_y = tf.matmul(x, y)  # Matrix product
x_mule_y = tf.multiply(x, y)
log_x = tf.log(x) 
x_equal_y = tf.equal(x, y)
x_less_y = tf.less(x, y)
x_great_eq_y = tf.greater_equal(x, y)

Variable initialization and evaluation

In [25]:
tf.global_variables_initializer().run()
x_plus_y_val = sess.run(x_plus_y)
x_mul_y_val = sess.run(x_mul_y)
x_mule_y_val = sess.run(x_mule_y)
log_x_val = sess.run(log_x)
x_equal_y_val = sess.run(x_equal_y)
x_less_y_val = sess.run(x_less_y)
x_great_eq_y_val = sess.run(x_great_eq_y)
print('''Plus: {plus}'''.format(plus = x_plus_y_val))
print('''Mul: {mul}'''.format(mul = x_mul_y_val))
print('''Mule: {mule}'''.format(mule = x_mule_y_val))
print('''Log: {log}'''.format(log = log_x_val))
print('''Equal: {eq}'''.format(eq = x_equal_y_val))
print('''Less: {less}'''.format(less =x_less_y_val))
print('''Great: {gre}'''.format(gre =x_great_eq_y_val))
sess.close()

Plus: [[5. 5.]
 [6. 6.]]
Mul: [[10.  7.]
 [24. 17.]]
Mule: [[4. 6.]
 [9. 8.]]
Log: [[0.        0.6931472]
 [1.0986123 1.3862944]]
Equal: [[False False]
 [ True False]]
Less: [[ True  True]
 [False False]]
Great: [[False False]
 [ True  True]]


Filter and reduce type operations

In [26]:
graph = tf.Graph()
sess = tf.InteractiveSession(graph=graph)

# x, y
x = tf.constant([[1, 2], [3, 4]])
y = tf.constant([[4, 3], [3, 2]])

# Condition
condition = tf.constant([[True, False], [True, False]],
                        dtype=tf.bool)
x_cond_y = tf.where(condition, x, y, name=None)

# Sum reduction
x_sum_0 = tf.reduce_sum(x,
                        axis=[0],
                        keepdims=False)
x_sum_1 = tf.reduce_sum(x,
                        axis=[1],
                        keepdims=False)
x_sum_2 = tf.reduce_sum(x, 
                       axis=[1],
                       keepdims=True)


Variable initialization and evaluation

In [27]:
tf.global_variables_initializer().run()
x_cond_y_val = sess.run(x_cond_y)
x_sum_0_val = sess.run(x_sum_0)
x_sum_1_val = sess.run(x_sum_1)
x_sum_2_val = sess.run(x_sum_2)
print('''Cond: {cond}'''.format(cond = x_cond_y_val))
print('''Sum 0: {s0}'''.format(s0 = x_sum_0_val))
print('''Sum 1: {s1}'''.format(s1 = x_sum_1_val))
print('''Sum 2: {s2}'''.format(s2 = x_sum_2_val))
sess.close()

Cond: [[1 3]
 [3 2]]
Sum 0: [4 6]
Sum 1: [3 7]
Sum 2: [[3]
 [7]]


There are two additional operations: **gather** and **scatter**. These are relevant since until recent times they were the only way we could index tensors. Scatter allows a value to be assigned to a given group of indexes, gather obtains tensor 'slices'. 


In [28]:
graph = tf.Graph()
sess = tf.InteractiveSession(graph=graph)

# Scatter
x = tf.Variable(tf.constant([1, 9, 3, 10, 5],
                            dtype=tf.float32,
                            name='satter_update'))
indices = [1, 3]
updates = tf.constant([2, 4],
                      dtype=tf.float32)
tf_scatter_update = tf.scatter_update(x,
                                      indices,
                                      updates,
                                      use_locking=None,
                                      name=None)


indices = [[0], [1], [3]]
updates = tf.constant([[0, 7, 0], [1, 1, 1], [2, 2, 2]])
shape = [4, 3]
tf_scatter_nd_2 = tf.scatter_nd(indices,
                                updates,
                                shape,
                                name=None)

# Gather

# 1d
params = tf.constant([1,2,3,4,5],dtype=tf.float32)
indices = [1,4]
tf_gather = tf.gather(params, indices, 
                      validate_indices=True, 
                      name=None)

# Nd
params = tf.constant([[0,0,0],[1,1,1],[2,2,2],[3,3,3]],dtype=tf. float32) 
indices = [[0],[2]] 
tf_gather_nd = tf.gather_nd(params, indices, name=None)

Variable initialization and evaluation

In [29]:
tf.global_variables_initializer().run()
tf_scatter_update_val = sess.run(tf_scatter_update)
tf_scatter_nd_2_val = sess.run(tf_scatter_nd_2)
tf_gather_val = sess.run(tf_gather)
tf_gather_nd_val = sess.run(tf_gather_nd)
print('''Scatter 1: {s1}'''.format(s1 = tf_scatter_update_val))
print('''Scatter 2: {s2}'''.format(s2 = tf_scatter_nd_2_val))
print('''Gather 1: {g1}'''.format(g1 = tf_gather_val))
print('''Gather 2: {g2}'''.format(g2 = tf_gather_nd_val))
sess.close()

Scatter 1: [1. 2. 3. 4. 5.]
Scatter 2: [[0 7 0]
 [1 1 1]
 [0 0 0]
 [2 2 2]]
Gather 1: [2. 5.]
Gather 2: [[0. 0. 0.]
 [2. 2. 2.]]


### Neural nets operations

Besides the regular vector operations, Tensorflow also provides a suite of tensorlike operations for working over neural networks, for example: convolutions and pooling. 

![alt text](./imgs/convolution.png "Convolucion")
![alt text](./imgs/pooling.png "Convolucion")


In [36]:
graph = tf.Graph()
sess = tf.InteractiveSession(graph=graph)

The name of the function that performs the convolutions ```tf.nn.conv2d``` and its parameters are: *input*, *filter*, *strides* y *padding*. 

- input : a four dimension tensor representing the input:  [batch_size, height, width, channels]
- filter: a four dimension tensor representing the convolution window: [height, width, in_channels, out_channels]
- stride: a four element list: [batch_stride, height_stride, width_stride, channels_stride]. 
- padding: it can be 'SAME' o 'VALID'. Specifies the behaviour of the filter near the edges

In [37]:
X = tf.constant([
    [
    [[1, 1], [2, 2], [3, 3], [4, 4]],
    [[4, 4], [3, 3], [2, 3], [1, 1]],
    [[5, 5], [6, 6], [7, 7], [8, 8]],
    [[8, 8], [7, 7], [6, 6], [5, 5]]
    ]
], dtype=tf.float32)

X_FILTER = tf.constant([
    [
        [[0.5], [0.5]], [[1], [1]]
    ],
    [
        [[0.5], [0.5]], [[1], [1]]
    ]
], dtype=tf.float32)


# Stride  batch, height, width, channel
X_STRIDE = [1, 1, 1, 1]
# Valid, Same... same padds with 0's
X_PADDING = 'VALID'

X_CONV = tf.nn.conv2d(
    input=X,
    filter=X_FILTER,
    strides=X_STRIDE,
    padding=X_PADDING
)

Pooling operation

In [38]:
# Batch lenght, height, width, channels
X_KSIZE = [1, 2, 2, 1]
X_STRIDE = [1, 2, 2, 1]

X_POOL = tf.nn.max_pool(
    value=X,
    ksize=X_KSIZE,
    strides=X_STRIDE,
    padding=X_PADDING
)

Variable initialization and evaluation

In [39]:
# EVAL
(X_FILTER_EVAL,
 X_EVAL,
 X_CONV_EVAL,
 X_POOL_EVAL) = sess.run([X_FILTER, X, X_CONV, X_POOL])

# Print
print('Filter')
print(X_FILTER_EVAL)
print('X')
print(X_EVAL)
print('Convolution')
print(X_CONV_EVAL)
print('Pooling')
print(X_POOL_EVAL)
#
sess.close()

Filter
[[[[0.5]
   [0.5]]

  [[1. ]
   [1. ]]]


 [[[0.5]
   [0.5]]

  [[1. ]
   [1. ]]]]
X
[[[[1. 1.]
   [2. 2.]
   [3. 3.]
   [4. 4.]]

  [[4. 4.]
   [3. 3.]
   [2. 3.]
   [1. 1.]]

  [[5. 5.]
   [6. 6.]
   [7. 7.]
   [8. 8.]]

  [[8. 8.]
   [7. 7.]
   [6. 6.]
   [5. 5.]]]]
Convolution
[[[[15. ]
   [16. ]
   [15.5]]

  [[27. ]
   [28. ]
   [27.5]]

  [[39. ]
   [39. ]
   [39. ]]]]
Pooling
[[[[4. 4.]
   [4. 4.]]

  [[8. 8.]
   [8. 8.]]]]
